<a href="https://colab.research.google.com/github/viridis45/Python-data-analysis/blob/master/20190828_6_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%99%80_%EC%8B%9C%ED%80%80%EC%8A%A4%EB%A5%BC_%EC%9C%84%ED%95%9C_%EB%94%A5%EB%9F%AC%EB%8B%9D(1_3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

케라스 창시자에게 배우는 딥러닝 _ 프랑소와 숄레
- NLP딥러닝이란 패턴 인식에 지나지 않는다
- 텍스트를 수치로 변환 -- 벡터화 : (단어, 문자, n-그램) == 토큰
- 원핫인코딩과 토큰임베딩
f


In [0]:
from keras.preprocessing.text import Tokenizer

samples = ["We ain't guiltless, Lord our witness.", "Pray forgiveness but don't turn around.", "Your kiss excites me, kiss indicts me time to leave this God forsaken town."]

tokenizer = Tokenizer(num_words = 1000) #가장 빈도가 높은 1000개의 단어만
tokenizer.fit_on_texts(samples) # 단어 인덱스 구축
sequences = tokenizer.texts_to_sequences(samples) #문자열을 정수 인덱스로

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary') # but why not to_categorical?

word_index = tokenizer.word_index
print('{:.2f} 개의 토큰을 찾았습니다'.format(len(word_index)))






print("\n=========해싱기법을 통한 원핫인코딩============")

dimensionality = 1000 # 1000개 이상의 단어가 있을 시 해싱 충돌
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality)) #3차원공간을 만들고

for i, sample in enumerate(samples): #샘플들에 정수태그를 부여하고
  for j, word in list(enumerate(sample.split()))[:max_length]: # 샘플을 단어단위로 잘라내어 만든 리스트에서 천개만 뽑아낸다
    index = abs(hash(word)) %dimensionality #랜덤한 정수 인덱스로
    results[i,j,index] = 1 #원하는 위치의 0만 1로 변환
    
    
 

24.00 개의 토큰을 찾았습니다


단어 임베딩이란 무엇인가요
- 원핫인코딩은 고차원 벡터를 리턴한다 (단어 총 갯수의 차원ㅋ)
- 단어 임베딩은 데이터로부터 학습. 태깅이랑 비슷하다고 보면 됨
- 학습시켜도 되고 미리 맹길어져있는것 가져와도 됨.
- ex) accurate 와 exact사이의 관계를 반영


임베딩 맹길기
1. (samples, sequence_length)를 받아서 --> (samples, sequence_length, embedding_dimensionality) 를 변환
2. 이것 또한 마찬가지로 weight가 랜덤하게 주어지고 역전파를 통해 업데이트됨
3. 연습: 1만개 단어 -- 8차원 임베딩 -> 정수 시퀀스를(2D) 임베딩 시퀀스로(3D) --> 다시 2d로 바꾸어 dense 훈련

In [0]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features) #만개단어만

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen = maxlen) # remember how you had to pad everything into same length? this is doing that automatically, how nice.
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen = maxlen)


In [0]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length = maxlen)) #have 10000words. randomly decided to have 8 dimensions. like how conv2d had input data, give the input length.
model.add(Flatten()) # woah already?

model.add(Dense(1, activation='sigmoid')) #why one tho
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


W0828 02:43:20.512517 140430620292992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 100us/step - loss: 0.6778 - acc: 0.5976 - val_loss: 0.6398 - val_acc: 0.6840
Epoch 2/10
20000/20000 [==============================] - 1s 59us/step - loss: 0.5612 - acc: 0.7441 - val_loss: 0.5374 - val_acc: 0.7270
Epoch 3/10
20000/20000 [==============================] - 1s 57us/step - loss: 0.4687 - acc: 0.7849 - val_loss: 0.5066 - val_acc: 0.7442
Epoch 4/10
20000/20000 [==============================] - 1s 59us/step - loss: 0.4231 - acc: 0.8071 - val_loss: 0.4941 - val_acc: 0.7552
Epoch 5/10
20000/20000 [==============================] - 1s 57us/step - loss: 0.3925 - acc: 0.8250 - val_loss: 0.4946 - val_acc: 0.7582
Epoch 6/10
20000/20000 [==============================] - 1s 57us/step - loss: 0.3681 - acc: 0.8381 - val_loss: 0.4972 - val_acc: 0.7540
Epoch 7/10
20000/20000 [==============================] - 1s 58us/step - loss: 0.3474 - acc: 0.8501 - val_loss: 0.5037 - val_acc

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test)
test_acc

# 하나의 Dense층을 훈련함 == 각 단어를 독립적으로 다룸
# 시퀀스 전체 특성을 알게하려면 임베딩 층 위에 순환 층이나 1d합성곱 필요

25000/25000 [==============================] - 0s 17us/step


0.75548

순환신경망, Recurrent Neural Network
- 이전 학습내용을 기억하며 정보를 처리
- 반복할 때 이전에 계산한 정보를 재사용함

In [0]:
# SimpleRNN 을 넘파이로 구현해보자

import numpy as np

timesteps = 100
input_features = 32
output_features = 64

inputs = np.random.random((timesteps, input_features)) # 그냥 난수
state_t = np.zeros((output_features, )) # 초기상태를 모두 0으로

w = np.random.random((output_features, input_features)) #2차원 랜덤수 가중치
u = np.random.random((output_features, output_features))
b = np.random.random((output_features, )) #1차원 랜덤수 가중치

successive_outputs = []
for input_t in inputs: #taking the row
  output_t = np.tanh(np.dot(w, input_t) + np.dot(u, state_t) + b) # state_t becomes the 'previous' value
  successive_outputs.append(output_t)
  
  state_t = output_t #current output becomes next computation's 'previous' value
  
final_output_sequence = np.stack(successive_outputs, axis=0)

In [0]:
#SimpleRNN_마지막 타임스텝의 출력만 받는 경우

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [0]:
# SimpleRNN, 전체 상태 시퀀스만 반환하는 경우

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN (32, return_sequences = True))
model.summary()

# tbh i see no difference 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [0]:
# and multiple return seq for middle layers, and the final layer만 마지막 출력 반환

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN (32, return_sequences = True))
model.add(SimpleRNN (32, return_sequences = True))
model.add(SimpleRNN (32, return_sequences = True))
model.add(SimpleRNN (32))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 32)                2080      
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 문제에 적용해보기

from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000
maxlen = 500
batch_size = 32

print('loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words = max_features)
print(len(input_train), 'train sequence')
print(len(input_test), 'test sequence')


input_train = sequence.pad_sequences(input_train, maxlen =  maxlen)
input_test = sequence.pad_sequences(input_test, maxlen = maxlen)
print('input_train size :', input_train.shape)
print('input_test size :', input_test.shape)

from keras.layers import Dense
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(input_train, y_train, epochs = 10, batch_size=128, validation_split = 0.2)

loading data...
25000 train sequence
25000 test sequence
input_train size : (25000, 500)
input_test size : (25000, 500)
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 19s 927us/step - loss: 0.6449 - acc: 0.6116 - val_loss: 0.4951 - val_acc: 0.7790
Epoch 2/10
20000/20000 [==============================] - 18s 888us/step - loss: 0.4118 - acc: 0.8226 - val_loss: 0.4730 - val_acc: 0.7724
Epoch 3/10
20000/20000 [==============================] - 17s 875us/step - loss: 0.2960 - acc: 0.8823 - val_loss: 0.3538 - val_acc: 0.8506
Epoch 4/10
20000/20000 [==============================] - 18s 879us/step - loss: 0.2166 - acc: 0.9168 - val_loss: 0.4070 - val_acc: 0.8404
Epoch 5/10
20000/20000 [==============================] - 18s 878us/step - loss: 0.1519 - acc: 0.9446 - val_loss: 0.4137 - val_acc: 0.8434
Epoch 6/10
20000/20000 [==============================] - 18s 876us/step - loss: 0.1014 - acc: 0.9647 - val_loss: 0.4244 - val_acc: 0.85

In [0]:
from nltk import word_tokenize, pos_tag, ne_chunk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
sentence = '홍길동이는 오늘 멀캠에 지각했습니다'
pos_tag(word_tokenize(sentence))
sentence2 = 'Gildong is studying at Mulcam in Seoul'
sentence2 = pos_tag(word_tokenize(sentence2))

ne_chunk(sentence2)

TclError: ignored

Tree('S', [Tree('GPE', [('Gildong', 'NNP')]), ('is', 'VBZ'), ('studying', 'VBG'), ('at', 'IN'), Tree('ORGANIZATION', [('Mulcam', 'NNP')]), ('in', 'IN'), Tree('GPE', [('Seoul', 'NNP')])])

In [0]:
from keras.datasets import reuters
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)


(xtrain, ytrain),(xtest, ytest) = reuters.load_data(num_words=None, test_split = 0.2)


In [0]:
from keras.utils import to_categorical
len(np.unique(ytrain)) # 태그가 46종류 있습니다


46